In [18]:
import numpy as np
import pandas as pd
import re
import nltk
import keras
import pickle
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from keras.models import load_model

In [19]:
model = load_model('chatbot_model.h5')

In [20]:
import json

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

To predict the class, we will need to provide input in the same way as we did while training. So we will create some functions that will perform text preprocessing and then predict the class.

In [76]:
lem = WordNetLemmatizer()

def clean_sentences(sentence):
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence_words = word_tokenize(sentence)
    sentence_words = [lem.lemmatize(word.lower()) for word in sentence_words if word not in set(stopwords.words('english'))]
    
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    sentence_words = clean_sentences(sentence)
    #print(sentence_words)
    #bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w==s:
                 # assign 1 if current word is in the vocabulary position
                bag[i] = 1
            if show_details:
                print("found in bag: %s" %w)
            
    return(np.array(bag))


def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    #model.fit(x, y ,.....): x:-A Numpy array (or array-like), or a list of arrays (in case the model has multiple inputs).
    #model will take list of list in the form of an array, eg,[[0,0,....0,1]]
    #output will also be same as the type of the input
    res = model.predict(np.array([p]))[0]
    #print(res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    #print(results)
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
   

In [77]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = np.random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [78]:
res = chatbot_response('Thank you so much for helping!')
res

'Any time!'

In [79]:
a = np.array([1,2,3])
print(a)
print(type(a))
print(np.array([a]))
print(type(np.array([a])))

[1 2 3]
<class 'numpy.ndarray'>
[[1 2 3]]
<class 'numpy.ndarray'>


Now we will code a graphical user interface. For this, we use the Tkinter library which already comes in python. We will take the input message from the user and then use the helper functions we have created to get the response from the bot and display it on the GUI. 

In [ ]:
from tkinter import *
def send():
    #The first part, "1.0" means that the input should be read from line one, character zero (ie: the very first character).
    #END is an imported constant which is set to the string "end". The END part means to read until the end of the text box
    #is reached. The only issue with this is that it actually adds a newline to our input. 
    #So, in order to fix it we should change END to end-1c.
    #The -1c deletes 1 character, while -2c would mean delete two characters, and so on.
    msg = EntryBox.get("1.0", "end-1c").strip()
    EntryBox.delete("0.0", END)
    
    if(msg != ''):
        ChatLog.config(state = Normal)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
base = Tk()
base.title("ChatBot :)")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()